In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install --upgrade tensorflow

In [3]:
import pandas as pd
import numpy as np

dataPath = '/content/drive/MyDrive/Soccer Player Injury Prediction/Datasets/Final Dataset/injuryFinalVTeamA.csv'
df = pd.read_csv(dataPath)
df

,daily_load,fatigue,mood,readiness,sleep_duration,sleep_quality,soreness,stress,injury_ts,weekly_load,...,illness,historical_injury_count,stress_readiness_interaction,team_performance_readiness_interaction,offensive_performance_readiness_interaction,defensive_performance_readiness_interaction,acwr_rolling_avg,stress_rolling_sum,injury,Player_name
0,0.0,3.0,4.0,6.0,8.0,4.0,3.0,0.75,0.0,0.000000,...,0,0,24.0,38.16,41.64,36.84,0.320870,10.50,0,TeamA-3e5f6e2b-46b7-4890-84a9-3bbb2649af5a
1,200.0,4.0,3.0,8.0,8.0,4.0,3.0,0.50,0.0,0.488536,...,0,0,24.0,56.40,53.92,53.20,0.260681,7.25,0,TeamA-af719df9-3e6c-4ad4-9e8e-0c0c45f4f76a
2,0.0,2.0,3.0,6.0,5.0,2.0,2.0,0.50,0.0,0.169312,...,0,0,18.0,44.58,26.46,34.38,0.250511,6.75,0,TeamA-b58af410-da77-479e-b93c-e03617b9f36d
3,0.0,3.0,3.0,7.0,8.0,4.0,3.0,0.50,0.0,0.000000,...,0,0,21.0,54.88,48.51,47.95,0.244236,7.00,0,TeamA-e920ae60-5c4b-4597-be27-fc6876dcec33
4,540.0,3.0,3.0,6.0,8.0,3.0,3.0,0.50,0.0,0.368607,...,0,3,18.0,36.30,36.48,41.76,0.340456,7.00,0,TeamA-5cd7a61b-88b2-46d2-94f8-5a0d4f682d93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7231,420.0,3.0,4.0,7.0,8.0,4.0,4.0,0.75,0.0,0.393298,...,0,0,28.0,53.69,48.16,47.53,0.217469,10.50,0,TeamA-c4ccf1a6-48c3-4a17-8d6c-eedd12e8680e
7232,320.0,2.0,3.0,5.0,7.0,4.0,2.0,0.50,0.0,0.440917,...,0,0,15.0,32.40,27.50,29.15,0.175310,7.00,0,TeamA-4051bba7-1170-4c43-b912-8c38815a7625
7233,0.0,3.0,4.0,6.0,8.0,3.0,2.0,0.25,0.0,0.176367,...,0,0,12.0,40.74,27.60,34.38,0.246456,3.50,0,TeamA-f20565cc-df22-46a7-aa97-af8ed00601d7
7234,0.0,3.0,3.0,7.0,8.0,4.0,3.0,0.50,0.0,0.000000,...,0,0,21.0,46.90,45.57,46.62,0.244719,7.00,0,TeamA-e920ae60-5c4b-4597-be27-fc6876dcec33


In [4]:
# Print all column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['daily_load', 'fatigue', 'mood', 'readiness', 'sleep_duration',
       'sleep_quality', 'soreness', 'stress', 'injury_ts', 'weekly_load',
       'Team', 'Total_distance', 'Average_running_speed', 'Top_speed', 'HIR',
       'acwr', 'atl', 'ctl28', 'monotony', 'strain', 'team_performance',
       'offensive_performance', 'defensive_performance', 'illness',
       'historical_injury_count', 'stress_readiness_interaction',
       'team_performance_readiness_interaction',
       'offensive_performance_readiness_interaction',
       'defensive_performance_readiness_interaction', 'acwr_rolling_avg',
       'stress_rolling_sum', 'injury', 'Player_name'],
      dtype='object')


In [5]:
#Defining class imbalce functions

#Oversampling
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

def oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply oversampling on the training data
    sampling_ratio = 0.35
    ros = RandomOverSampler(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100
    #print(f"Percentage of 0s after oversampling: {zero_percentage_after:.2f}%")

    return X_train_resampled, y_resampled, zero_percentage_after


# Undersampling
def undersample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply undersampling on the training data
    sampling_strategy = 1 - sampling_ratio
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after undersampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100

    return X_train_resampled, y_resampled, zero_percentage_after


#SMOTE
def smote_data(X_train, y_train, sampling_ratio):

    # Set a custom sampling ratio
    sampling_ratio = 0.35

    # Apply SMOTE with the custom ratio
    smote = SMOTE(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    # Reshape the resampled data
    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


# ADASYN
def adasyn_oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply ADASYN oversampling on the training data
    sampling_strategy = sampling_ratio
    adasyn = ADASYN(sampling_strategy=sampling_strategy, random_state=42, n_neighbors=5)
    X_resampled, y_resampled = adasyn.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


#Data sample mode
def sample_mode(X_train, y_train, sampling_ratio, oversample_mode):
 precentageOfZeroesInDataset = (y_train == 0).sum() / len(y_train)

 if oversample_mode == 'oversample':
      X_train, y_train, precentageOfZeroesInDataset = oversample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'undersample':
      X_train, y_train, precentageOfZeroesInDataset = undersample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'smote':
      X_train, y_train, precentageOfZeroesInDataset = smote_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'adasyn':
      X_train, y_train, precentageOfZeroesInDataset = adasyn_oversample_data(X_train, y_train, sampling_ratio)


 return X_train, y_train, precentageOfZeroesInDataset

In [6]:
#Confusion matrtix that shows injuries as TP
def confusion_matrix_only_injuries(y_test, y_pred):
    tp, fn, fp, tn = 0, 0, 0, 0
    for i, j in zip(y_test, y_pred):
        if i == 1 and j == 1:
            tp += 1
        elif i == 1 and j == 0:
            fn += 1
        elif i == 0 and j == 1:
            fp += 1
        elif i == 0 and j == 0:
            tn += 1
    # Create a DataFrame for the confusion matrix
    confusion_matrix = pd.DataFrame(
        {
            "Predicted Injuries": [tp, fp],
            "Predicted Non-Injuries": [fn, tn],
        },
        index=["Actual Injuries", "Actual Non-Injuries"]
    )

    return confusion_matrix

In [7]:
# Datapreprocessing

def create_player_sequences(df, player, sequence_length=7):
    player_df = df[df['Player_name'] == player]
    # player_data = player_df.drop(['Date', 'Player_name'], axis=1).values
    player_data = player_df.drop(['Player_name'], axis=1).values

    sequences = []
    for i in range(len(player_data) - sequence_length):
        sequences.append(player_data[i:i + sequence_length])

    return np.array(sequences)

def create_team_sequence(df, sequence_length):
    players = df['Player_name'].unique()
    all_sequences = []
    for player in players:
        player_sequences = create_player_sequences(df, player, sequence_length)
        all_sequences.append(player_sequences)

    data = np.concatenate(all_sequences, axis=0)
    X, y = data[:, :-1, :], data[:, -1, -1]
    return X, y


def preprocess(inputWindow, interpolate=False):
    # Load your dataframe
    # df = merge_data_for_team("TeamA", interpolate=interpolate)
    df = pd.read_csv(dataPath)

    # # Convert 'Duration' column to seconds
    # df['Duration'] = df['Duration'].apply(convert_duration_to_seconds)

    # Preprocess the data
    X, y = create_team_sequence(df, inputWindow)

    # Convert data types
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    return X, y

In [8]:
# #LSTM model

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# def lstm(test_size, oversample_mode, sampling_ratio, epochs = 50):
#     def create_lstm_model(input_shape):
#         model = Sequential()
#         model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
#         model.add(Dropout(0.2))
#         model.add(LSTM(32))
#         model.add(Dropout(0.2))
#         model.add(Dense(1, activation='sigmoid'))

#         model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#         return model

#     input_shape = (X.shape[1], X.shape[2])
#     model = create_lstm_model(input_shape)

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

#     X_train, y_train, precentageOfZeroesInDataset = sample_mode(X_train, y_train, sampling_ratio, oversample_mode)

#     # Train the model
#     history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

#     # Evaluate the model
#     loss, accuracy = model.evaluate(X_test, y_test)

#     # Get the predicted labels
#     y_pred_probs = model.predict(X_test)
#     y_pred = (y_pred_probs > 0.5).astype(int)

#     # Calculate evaluation metrics
#     f1 = f1_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     confInjuries = confusion_matrix_only_injuries(y_test, y_pred)

#     return loss, accuracy, f1, precision, recall, confInjuries, precentageOfZeroesInDataset

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import numpy as np

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="`use_label_encoder` is deprecated in 1.7.0.")

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a list to store the results
results = []

# List of hyperparameters
input_windows =  [2, 4, 8, 16, 32]
test_sizes = [0.2, 0.3]
oversample_modes = ["none", "oversample", "undersample", "smote", "adasyn"]
sampling_ratios = [0.2, 0.3]
interpolate_injuries = [True, False]

# Iterate through all combinations of hyperparameters
for interploate_injuries in interpolate_injuries:
    for input_window in input_windows:
        X, y = preprocess(input_window, interpolate=interploate_injuries)

        for test_size in test_sizes:
            for oversample_mode in oversample_modes:
                for sampling_ratio in sampling_ratios:

                    input_shape = (X.shape[1], X.shape[2])
                    model = create_lstm_model(input_shape)

                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

                    X_train, y_train, precentageOfZeroesInDataset = sample_mode(X_train, y_train, sampling_ratio, oversample_mode)

                    # Train the model
                    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

                    # Evaluate the model
                    loss, accuracy = model.evaluate(X_test, y_test)

                    # Get the predicted labels
                    y_pred_probs = model.predict(X_test)
                    y_pred = (y_pred_probs > 0.5).astype(int)

                    # Calculate evaluation metrics
                    f1 = f1_score(y_test, y_pred)
                    precision = precision_score(y_test, y_pred)
                    recall = recall_score(y_test, y_pred)
                    #confInjuries = confusion_matrix_only_injuries(y_test, y_pred) # define this function

                    # Save the results
                    results.append({"input_window": input_window,
                                    "interpolate_injuries": interpolate_injuries,
                                    "test_size": test_size,
                                    "oversample_mode": oversample_mode,
                                    "sampling_ratio": sampling_ratio,
                                    "Prectenage of zeroes in dataset": precentageOfZeroesInDataset,
                                    "accuracy": accuracy,
                                    "f1": f1,
                                    "precision": precision,
                                    "recall": recall,
                                    #"confusion": confInjuries
                                    })

# Sort the results based on the F1 score
results.sort(key=lambda x: x['f1'], reverse=True)

# Print the top 3 performers
for i in range(min(3, len(results))):
    print(f"Top Performer {i + 1}:")
    print(f"Input window: {results[i]['input_window']}")
    print(f"Interpolate injuries: {results[i]['interpolate_injuries']}")
    print(f"Test Size: {results[i]['test_size']}")
    print(f"Oversample Mode: {results[i]['oversample_mode']}")
    print(f"Sampling Ratio: {results[i]['sampling_ratio']}")
    print(f"Percentage of zeroes in dataset: {results[i]['Prectenage of zeroes in dataset']}")
    print(f"Accuracy: {results[i]['accuracy']}")
    print(f"F1: {results[i]['f1']}")
    print(f"Precision: {results[i]['precision']}")
    print(f"Recall: {results[i]['recall']}")
    #display(results[i]['confusion'])
    print("\n")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
91/91 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.8658 - loss: 0.4437 - val_accuracy: 0.9924 - val_loss: 0.0562
Epoch 2/2
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9925 - loss: 0.0528 - val_accuracy: 0.9924 - val_loss: 0.0448
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9919 - loss: 0.0479
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7645 - loss: 0.4958 - val_accuracy: 0.9924 - val_loss: 0.0550
Epoch 2/2
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9927 - loss: 0.0536 - val_accuracy: 0.9924 - val_loss: 0.0442
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9919 - loss: 0.0474
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

99/99 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.7024 - loss: 0.5449 - val_accuracy: 0.9507 - val_loss: 0.1698
Epoch 2/2
99/99 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.9076 - loss: 0.2555 - val_accuracy: 0.9531 - val_loss: 0.1632
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9558 - loss: 0.1586 
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.8345 - loss: 0.4568 - val_accuracy: 0.9359 - val_loss: 0.1653
Epoch 2/2
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.8984 - loss: 0.2630 - val_accuracy: 0.9354 - val_loss: 0.1545
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9431 - loss: 0.1414
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.7507 - loss: 0.5147 - val_accuracy: 0.9793 - val_loss: 0.0905
Epoch 2/2
96/96 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.9066 - loss: 0.2644 - val_accuracy: 0.9546 - val_loss: 0.1462
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9606 - loss: 0.1399
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
Top Performer 1:
Input window: 32
Interpolate injuries: [True, False]
Test Size: 0.2
Oversample Mode: oversample
Sampling Ratio: 0.3
Percentage of zeroes in dataset: 74.07458563535911
Accuracy: 0.9312130212783813
F1: 0.1467889908256881
Precision: 0.08080808080808081
Recall: 0.8


Top Performer 2:
Input window: 4
Interpolate injuries: [True, False]
Test Size: 0.2
Oversample Mode: adasyn
Sampling Ratio: 0.2
Percentage of zeroes in dataset: 83.51906158357771
Accuracy: 0.9902507066726685
F1: 0.12500000000000003
Precision: 0.2
Recall: 0.09090909090909091


Top Performer 3:
Input window: 32
Interpolate injuries: [True, False]
Test S